In [1]:
import pandas as pd
import tensorflow as tf
import lib.dataProcess as dp
import lib.deepLearning as dl

In [2]:
raw_data = pd.read_csv('./GEO_input_ensemble_CV_500.csv')

In [3]:
shuffled_data = raw_data
shuffled_data['index'] = dp.shuffle_index(shuffled_data)

In [4]:
fivefold = dp.n_fold(raw_data, 'index', 5)

In [5]:
xdata_five, ydata_five = dp.divide_xy_train(fivefold, 'result', True, 1, -3)
train_x, test_x = dp.train_test(xdata_five, 0)
train_y, test_y = dp.train_test(ydata_five, 0)
train_y = dp.one_hot_encoder(train_y)
test_y = dp.one_hot_encoder(test_y)
val_x, test_x = dp.test_validation(test_x)
val_y, test_y = dp.test_validation(test_y)

Number of Class : 2
Number of Class : 2


In [6]:
nodes = [200,100, 50]
learning_rate = 0.001
batch_size = 100

X, Y, layers, logits, phase, hypothesis, cost, train, predicted, correct_prediction, accuracy, keep_prob = dl.set_model_dropout(train_x, train_y, nodes , learning_rate)



In [7]:

best_train_acc = 0
best_val_acc = 0
best_test_acc = 0 
best_cost = float("inf")
save_path_dir ='./'
count = 0
model_num = 0

saver = tf.train.Saver()
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
    stop_switch = True
    step = 0

    while stop_switch:
        total_num = int(len(train_x) / batch_size)
        for k in range(total_num):
            batch_x = train_x[k * batch_size:(k + 1) * batch_size]
            batch_y = train_y[k * batch_size:(k + 1) * batch_size]
            sess.run(train, feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.5 , phase:True})


        train_h, train_c, train_p, train_a = sess.run( [hypothesis, cost, predicted, accuracy], feed_dict={X: train_x, Y: train_y, keep_prob: 1 , phase:False})
        val_h, val_c, val_p, val_a = sess.run([hypothesis, cost, predicted, accuracy], feed_dict={X: val_x, Y: val_y, keep_prob: 1 , phase:False})
        if step % 20 == 0 :
            print("train acc : ", train_a, "validation acc : ", val_a, "train_cost", train_c)
        step += 1

        if best_cost > val_c :
            best_train_acc = train_a
            best_val_acc = val_a
            best_cost = val_c
            count = 0
            save_path = saver.save(sess, save_path_dir + 'model'+str(model_num)+'.ckpt')

        elif count > 10 :
            stop_switch = False
            print("Learning Finished!! \n")
        else:
            count += 1

    print("Training Accuracy : ", best_train_acc,  "Validation Accuracy : ", best_val_acc)

    saver.restore(sess, save_path)

    test_h, test_p, test_a = sess.run([hypothesis, predicted, accuracy],
                                      feed_dict={X: test_x, Y: test_y, keep_prob:1.0 , phase:False})
    print("\nTest Accuracy: ", test_a)
    best_test_acc = test_a


    model_num += 1                    
    

train acc :  0.850555 validation acc :  0.854586 train_cost 0.325528
train acc :  0.990891 validation acc :  0.957494 train_cost 0.0183502
Learning Finished!! 

Training Accuracy :  0.984913 Validation Accuracy :  0.966443
INFO:tensorflow:Restoring parameters from ./model0.ckpt

Test Accuracy:  0.966518


In [7]:
data_x, data_y = dp.divide_xy_test(raw_data, 'result', 1,-3)
ids = raw_data['patient']

In [8]:
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, './model0.ckpt')
    test_h, test_p = sess.run([hypothesis, predicted], feed_dict={X: data_x, Y: dp.one_hot_encoder(data_y), keep_prob:1.0 , phase:False})

INFO:tensorflow:Restoring parameters from ./model0.ckpt
Number of Class : 2


In [9]:
dp.prediction_probability(test_h, test_p, data_y, ids)

In [12]:
len(test_h)

4408

In [13]:
len(data_y)

4408

In [15]:
len(ids)

4408

In [17]:
len(test_p)

4408

In [18]:
test_p

array([0, 1, 1, ..., 1, 1, 1])

In [19]:
test_h

array([[  9.78414536e-01,   2.15854533e-02],
       [  1.48151221e-03,   9.98518527e-01],
       [  1.59435826e-10,   1.00000000e+00],
       ..., 
       [  9.48166323e-10,   1.00000000e+00],
       [  3.21645871e-06,   9.99996781e-01],
       [  2.25224676e-06,   9.99997735e-01]], dtype=float32)